In [1]:
import yolort

In [2]:
import argparse
import platform
import sys
import time
from pathlib import Path

In [3]:
import pandas as pd

In [4]:
FILE = Path().resolve()
ROOT = FILE.parents[0]  # YOLOrt root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH

### 配置环境

In [5]:
import os
import cv2
import numpy

import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # PCI_BUS_ID” # 按照PCI_BUS_ID顺序从0开始排列GPU设备 
os.environ["CUDA_VISIBLE_DEVICES"]="0"  #设置当前使用的GPU设备仅为0号设备  设备名称为'/gpu:0'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cuda = device.type != 'cpu'

### 导入模型和定义的函数

In [6]:
from yolort.models.yolo import YOLO
from yolort.models import YOLOv5
from yolort.utils import Visualizer, get_image_from_url, read_image_to_tensor, check_dataset
from yolort.v5 import load_yolov5_model, letterbox, non_max_suppression, attempt_download
from yolort.v5.utils.downloads import safe_download
from yolort.v5.utils.dataloaders import *
from yolort.v5.utils.general import colorstr, increment_path


from yolort.data.metric import cal_benchmark

### 加载数据

In [7]:
data = ROOT / 'data/coco128.yaml'  # dataset.yaml path
data = check_dataset(data)

In [8]:
img_size = 640
stride = 32
score_thresh = 0.25
batch_size = 32
nms_thresh = 0.45
single_cls=False # treat as single-class dataset
rect = False
workers=8  # max dataloader workers (per RANK in DDP mode)

In [9]:

task = 'val'
task = task if task in ('train', 'val', 'test') else 'val'  # path to train/val/test images
loader,dataset = create_dataloader(data[task],
                               img_size,
                               stride,
                               batch_size,
                               single_cls,
                               rect=rect,
                               workers=workers,
                               prefix=colorstr(f'{task}: '))


val: Scanning '/home/guest1/LISHUFEI/jupyter_code/datasets/coco128/labels/train2017.cache' images and labels... 128 found


### 加载模型权重

In [10]:
model_path = 'yolov5s.pt'
checkpoint_path = attempt_download(model_path)

In [11]:
project = ROOT / 'runs/val',  # save to project/name
name='exp',  # save to project/name


### 加载yolov5模型

In [12]:
model_yolov5 = load_yolov5_model(checkpoint_path)
model_yolov5 = model_yolov5.to(device)
model_yolov5 = model_yolov5.eval()

In [13]:
cal_benchmark(data,
              batch_size,
              img_size,
              score_thresh,
              nms_thresh,
              model_yolov5,
              loader, 
              device,
              single_cls,
              is_v5=True
             )

               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:   0%|          | 0/4 [00:00<?, ?i/home/guest1/anaconda3/envs/pytorch-lsf/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 4/4 [00:10<00:00
                 all        128        929      0.709      0.604      0.695      0.498
Speed: 0.2ms pre-process, 9.7ms inference, 9.2ms NMS per image at shape (32, 3, 640, 640)


### 加载yolort模型

In [14]:
model_yolort = YOLO.load_from_yolov5(
    checkpoint_path,
    score_thresh=score_thresh,
    nms_thresh=nms_thresh,
    version="r6.0",
)
model_yolort = model_yolort.eval()
model_yolort = model_yolort.to(device)

In [15]:
cal_benchmark(data,
              batch_size,
              img_size,
              score_thresh,
              nms_thresh,
              model_yolort,
              loader, 
              device,
              single_cls,
              is_v5=False
             )

               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 4/4 [00:09<00:00
                 all        128        929      0.733      0.632      0.691        0.5
Speed: 0.2ms pre-process, 15.3ms inference, 0.0ms NMS per image at shape (32, 3, 640, 640)
